![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Logo_of_Hochschule_Kaiserslautern.png/320px-Logo_of_Hochschule_Kaiserslautern.png)

![](https://miro.medium.com/v2/resize:fit:640/format:webp/0*mmUHzD_vxDJn5Rfc.gif)

# **Eine Einfuehrung in die Objekterkennung**

Trainieren und anwenden von künstlichen neuronalen Netzen.

Ein Workshop von *Max*.

# **1. Einrichten der Umgebung**

- Repository klonen
- Abhänigkeiten installieren *(mit `pip3.x`)*
- Installation Prüfen: *print PyTorch* und *GPU Version*

In [ ]:
!git clone https://github.com/maja1001/yolov5.git

%cd yolov5
%pip install -r requirements.txt
%pip install roboflow

print("\n[Info] Repository erfolgreich geklont und Abhänigkeiten installiert.")

In [ ]:
import torch

print( f"[Info] Torch Version: {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'Keine Grafikkarte gefunden: CPU wird verwendet'})" )

In [ ]:
!git pull

# **2. Verifizierung der Codebasis**

- Validiere die Installation des Repositories anhand von Test Bildern / Detektionen
- Einsatz von `yolov5s`
- Alternative auch:
  - `yolov5n`
  - `yolov5m`
  - `yolov5l`
  - `yolov5x`

In [ ]:
# Teste detect
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/bus.jpg --name test/bus
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/zidane.jpg --name test/zidane

!python detect.py --weights yolov5s.pt --img 640 --conf 0.7 --source data/images/bus.jpg --name test/busHighConf
!python detect.py --weights yolov5s.pt --img 640 --conf 0.7 --source data/images/zidane.jpg --name test/zidaneHighConf

# test both pictures
# !python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

In [ ]:
# Entferne bisherige Ergebnisse
!rm -r runs/detect/test

# **3. Datensatz einbinden**
- Under der Verwendung des Roboflow Packages
- Roboflow Schnittstelle mit entsprechenden Schlüssel nutzen
- Hier geht es zu [Roboflow](https://roboflow.com/?ref=ultralytics)

Hier geht es zu einigen Vorbereiteten [Datensätzen](https://universe.roboflow.com/hsklworkshop-m8g4f)

In [ ]:
# Datensatz einbinden
from roboflow import Roboflow

rf = Roboflow(api_key="vB3ieGw7uAwm1qyRpILr")
project = rf.workspace("hsklworkshop-m8g4f").project("laptop-o9ebb")

# Roh
dataset = project.version(1).download("yolov5")

# Mit Data Augmentation
dataset = project.version(2).download("yolov5")

# **4. Training**

### **Es gibt mehrere Möglichkeiten ein Training durchzuführen**

1. In einem Google Colaboratory (Colab)
2. Alternative über das [Ultralytics HUB](https://hub.ultralytics.com/home)

#### **1. Google Colab**

- Basis Befehl / Python Script: `train.py`
- Hilfe under: `train.py --help`
- Mehrere Argumente sind möglich:

```
# Auszug
--img 320/512/640/..
--batch 16/32/64/128
--epochs
--cfg /path/to/_.yaml
--data /path/to/data.yaml
--weights /path/to/_.pt
--hyp /path/to/_.yaml
--project /dataset/train_journey/myFirstTraining
--name myFirstTraining
--cache ram
```

- Default Werte werden gesetzt
- Beispielhafter Trainings Befehl: `!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache ram`
- Einsatz von verschiedenen Trainingsmethoden möglich
  - Transfer Learning mit angehaltenen Schichten: ``` --freeze 10 ```



In [ ]:
# Mit MS COCO Beispiel
!python train.py xxx

In [ ]:
!python train.py --help

In [ ]:
# Individueller Datensatz
# !python train.py --img --batch --epochs --cfg --data --weights --hyp --project name/of/dataset/result --name --cache ram

In [ ]:
# Individueller Datensatz (laptop-1)


In [ ]:
# Individueller Datensatz - mit angehaltenen Schichten (freeze layers) (laptop-1)


In [ ]:
# Entferne Trainings Ergebnisse
!rm -r laptop-1/result

#### **2. Ultralytics HUB**

- Ultralytics HUB mit Roboflow verlinken
- Under Projekte "Create Model" wählen
- Colab mit HUB verlinken

In [ ]:
# Training via Ultralytics HUB
%pip install -U ultralytics
from ultralytics import YOLO, checks, hub
checks()

In [ ]:
# Training via Ultralytics HUB
hub.login('867d2f58d7c18fd4431f9edffa7e7995d1c5d23282')


model = YOLO('https://hub.ultralytics.com/models/TZs3shmNgxufMsJdfgij')
results = model.train()

# **5. Detektion und Test**
Es gibt mehrere Möglichkeiten das Model zu testen

- Local mithilfe von `detect.py`
- Local mithilfe des Roboflow Packages
- Auf der [Roboflow](https://roboflow.com/?ref=ultralytics) Webseite
- Auf der [UltralyticsHub](https://hub.ultralytics.com/) Webseite

In [ ]:
# Entferne bisherige Testergebnisse
!rm -r runs/detect

In [ ]:
# Local mithilfe von `detect.py`: Vortrainiert mit ms coco
!python detect.py --weights yolov5s.pt --img 640 --conf 0.2 --source data/images/smartphone_test.jpg --name smartphone_test_pretrained

In [ ]:
# Local mithilfe von `detect.py`: Smartphone
!python detect.py --weights trainedModels/3479_w_aug/weights/best.pt --img 640 --conf 0.5 --iou 0.35 --source data/images/smartphone_test.jpg --name smartphone_test

In [ ]:
# Local mithilfe von `detect.py`: bottle paper carton
!python detect.py --weights trainedModels/b_p_c_m_w_aug_640_run_1/weights/best.pt --img 640 --conf 0.4 --source data/images/b_p_c_test_1.jpg --name b_p_c_test_1
!python detect.py --weights trainedModels/b_p_c_m_w_aug_640_run_1/weights/best.pt --img 640 --conf 0.4 --source data/images/b_p_c_test_2.jpg --name b_p_c_test_2
!python detect.py --weights trainedModels/b_p_c_m_w_aug_640_run_1/weights/best.pt --img 640 --conf 0.4 --source data/images/b_p_c_test_3.jpg --name b_p_c_test_3

In [ ]:
# Local mithilfe von `detect.py`: Laptop
!python detect.py --weights trainedModels/laptop_raw_run_2_colab/weights/best.pt --img 640 --conf 0.4 --iou 0.35 --source data/images/laptop.jpg --name laptop
# !python detect.py --weights trainedModels/laptop_raw_run_1_ultralytics/weights/best.pt --img 640 --conf 0.4 --iou 0.35 --source data/images/laptop.jpg --name laptop

In [ ]:
# Local mithilfe von `detect.py`: Smartphone VIDEO
!python detect.py --weights trainedModels/3479_w_aug/weights/best.pt --img 640 --conf 0.4 --source data/videos/smartphone_test.mp4 --name smartphone_test_w_aug
!python detect.py --weights trainedModels/379_raw_run_1/weights/best.pt --img 640 --conf 0.4 --source data/videos/smartphone_test.mp4 --name smartphone_test_raw
!python detect.py --weights yolov5s.pt --img 640 --conf 0.4 --source data/videos/smartphone_test.mp4 --name smartphone_test_pretrained

# **6. Export**
- Basis Befehl / Python Script = `export.py`
- Mehrere Argumente sind möglich:

```
# Auszug
  --data
  --weights
  --imgsz
  --batch-size
  --device
  --half
    [...]
  --int8
  --workspace
  --nms
  --iou-thres
  --conf-thres
  --include
```

- Default Werte sind gesetzt
- Beispielhafter Export Befehl: `!python export.py --weights path/to/weights --include tflite --imgsz 640 640 --data /path/to/data.yaml`
- Visualisierung mithilfe von [Netron](https://netron.app/)

In [ ]:
# Export Beispiel
!python export.py --help

In [ ]:
# Export Beispiel
input_width = 640 #@param {type:"slider", min:64, max:4096, step:64}
input_height = 640 #@param {type:"slider", min:64, max:4096, step:64}

!python export.py --weights /path/to/_.pt --img {input_height} {input_width} --include tflite

# **7. Weitere Links**

- [Ultralytics Hub Workshop](https://hub.ultralytics.com/projects/PFGKwK1dCBJPguNS72k7)